# TUMOR CELL
Class representing tumor cells in the model.
Tumor cells can be cancerous or non-cancerous. Cancerous cells can induce endothelial growth, 
migrate, proliferate, and interact with endothelial cells.

Attributes:
    unique_id: The unique identifier for the agent.
    position: The position of the agent in the model grid.
    model: The model in which the agent resides.
    viable: Indicates if the tumor cell is viable.
    proliferation_prob: Probability of tumor cell proliferation.
    migration_prob: Probability of tumor cell migration.
    death_prob: Probability of tumor cell death.
    lifespan: Lifespan of the tumor cell (in steps).
    qs_max: Maximum oxygen concentration.
    qs: Current oxygen concentration.
    Ks_substrate: Substrate concentration for growth.
    mu_max: Maximum growth rate.
    Ks_growth: Growth rate constant.
    hypoxia_thresholds: Thresholds for different levels of hypoxia.
    nearest_endo: The nearest endothelial cell.
    nearest_dist: The distance to the nearest endothelial cell.
    prev_dist: The previous distance to the nearest endothelial cell.

## Imports and Class Declaration
Import the necessary prerequisites for this class to run as intended.

**Class Declaration**
class Tumor_cells(Agent): 

In [ ]:
from mesa import Agent;
import random

## Instance Initiation
Instance initalization associates the instance with different attribute parameter values depending on initialization conditions. For instance; whether an instance is created during model initiation or during tumor proliferation.


**Declaration**


def __init__(self, unique_id, position, model, *args):


Initialize a tumor cell with the given parameters.


Args:
    unique_id: Unique identifier for the agent.
    position: The position of the agent on the grid as a tuple (x, y).
    model: The model to which the agent belongs.
    *args: Optional arguments, including the previous distance to the nearest endothelial cell.


**Attributes**


There are 8 groups of instance attributes and parameters that relate to different components of tumor cells. These are constructor (1) initiation attributes, (2) basic parameter values, (3) kinetic parameters, (4) hypoxia parameters, (5) parameters related to endothelial cell tracking, (6) tumor cell-endothelial cell interaction parameters, (7) age related parameters, and (8) hunger related parameters. 


Additionally, instance initialization anchors the random number generator to a specified seed inorder to ensure similar simulation conditions for every simulation. 


In [ ]:
#Constructor
def __init__(self, unique_id, position, model, *args): #Position inputed as (x,y)

    super().__init__(unique_id, model);
    self.model = model;
    self.unique_id = unique_id;
    self.position = position;
    self.viable = True

    #SET RANDOM SEED
    random.seed(5)
    
    #BASICS
    self.proliferation_prob = 0.0846
    self.migration_prob = 0.1167 
    self.death_prob = 0.00284
    self.initial_resist_M1_prob = 0
    self.resistance_M1_prob = 0.004
    self.lifespan = 150 #steps
    
    #KINETIC PARAMETERS:
    self.qs_max = 30 #µmol oxygen
    self.qs = 0
    self.Ks_substrate = 5 #according to GPT oxygen
    self.mu_max = 0.06 #
    self.Ks_growth = 1000/(self.model.grid.width*self.model.grid.width) #µmol/L
    
    #HYPOXIA PARAMETERS
    self.max_signal_dist = 20
    self.optimal_signal_dist = 12
    self.hypoxia_thresholds = [2.0, 7.0, 25.0];

    #ENDO TRACKINGs
    self.nearest_endo = None;
    self.nearest_dist = None;
    self.prev_dist = None;
    if len(args) > 0:
        mothers_nearest_dist = args[0]
        self.prev_dist = mothers_nearest_dist #previous distance set
        self.set_nearest_endo() #identify nearest endo during initialization
    elif len(args) == 0:
        self.set_nearest_endo('default')
        
    self.diff = self.prev_dist - self.nearest_dist #This only works if set_nearest_endo is initialized.
    #print(f'Prev dist: {self.prev_dist}, Curr dist: {curr_dist}')
    if self.diff != 0:
        self.diff_sign = self.diff/abs(self.diff)
    else:
        self.diff_sign = 0
    
    #INTERACTION PARAMETERS (Distance Depen dent) -> self.tumor_endo_interaction() (NOT STANDARDIZED RANGES)
    self.death_intensity = 1.7  #1.7
    self.prolif_inhib_intensity = 0.4#0.04
    self.angiogenesis_intensity = 0.08 #0.8 #Keep it smaller for beutiful branching
    self.optimal_signal_dist_significane = 0.0 #0.01

    #AGE PARAMETERS
    self.recParam1 = 0

    #HUNGER PARAMETERS:
    self.prolif_inhibition = 0
    self.intensify_death = 0
    self.migration_inhibition = 0

## Setters 
Setters are public instnace methods called upon by the tumor cell agent as well as other agents inroder to comprehensivly adjust or directly set instance parameter values. 


*See docstrings for details*


Tumor_cells declare five setters, that vary in complexity. 

set_nearest_endo() Uses a relativley complex algorithm to identify and set the nearest endothelial cell to an Tumor_cell instance. 

set_proliferation_prob() & set_death_prob() Both methods use similar algorithms to adjust proliferation_prob and death_prb respectively based on method input. 

set_optimal_signal-dist(), set_angiogenesis_intesity() set_angiogeneisis_intensity() simply sets values for their respective parameters based on the input argument. 

In [ ]:
#SETTERS
def set_nearest_endo(self, *args):
    """
    Set the nearest endothelial cell to the tumor cell.

    Args:
        *args: Optional argument for default behavior.
    
    Returns:
        nearest_endo: The nearest endothelial agent.
        nearest_dist: The distance to the nearest endothelial agent.
    """
    endothelial_agents = self.model.endothelial_list; #endothelial_list is a list containing class type objects as elements.
    #print(f'Endothelial List: {self.model.endothelial_list}')
    counter = 1
    nearest_endo = None                     # initiate variable
    nearest_dist = None                     # initiate variable
    #FIND NEAREST ENDO
    for agent in endothelial_agents:
        x_other, y_other = agent.position
        x_self, y_self, = self.position
        distance_i = ((y_other-y_self)**2 + (x_other-x_self)**2)**(1/2)

        if nearest_endo is None or distance_i < nearest_dist:
            nearest_endo = agent
            nearest_dist = distance_i
            #   print(f'nearest dist {nearest_dist}')
        
        counter += 1
    
    #update new endo and dist
    self.nearest_endo = nearest_endo
    self.nearest_dist = nearest_dist
    
    #RECORD CURR DIST
    if len(args) > 0 and args[0] == 'default':
        #print("HEJ HEJ")
        self.prev_dist = self.nearest_dist

    return nearest_endo, nearest_dist

def set_proliferation_prob(self, *args):
    """
    Set the proliferation probability for the tumor cell.

    Args:
        *args: Can specify a value and type (proportion or value).
    """
    if args[0] == 'default':
        self.proliferation_prob = 0.0846
    elif args[0] != 'default':
        val = args[0]
        type = args[1]
        
        if type == "proportion":
            self.proliferation_prob = self.proliferation_prob * val
        if type == "value":
            self.proliferation_prob = val
        else:
            pass

def set_death_prob(self, *args):
    """
    Set the death probability for the tumor cell.

    Args:
        *args: Can specify a value and type (proportion or value).
    """
    if args[0] == "default":
        self.death_prob = 0.00284
    else:
        val = args[0]
        type = args[1]
        
        if type == "proportion":
            self.death_prob = self.death_prob * val
        if type == "value":
            self.death_prob = val
        else:
            pass
    pass  

def set_optimal_signal_dist(self, *args):
    """
    Set the optimal signaling distance for angiogenesis.

    Args:
        *args: A single argument specifying the optimal distance.
    """
    self.optimal_signal_dist = args[0]

def set_angiogenesis_intensity(self, *args):
    """
    Set the intensity of angiogenesis (blood vessel growth).

    Args:
        *args: A single argument specifying the intensity.
    """
    if len(args) == 1:
        self.angiogenesis_intensity = args[0]

## Data display
*print_agent_data()* can be called to print instance data as specified. 
Includes information about the tumor's age, proliferation probability,
death probability, direction sign, and distance to the nearest endothelial cell.

**Declaration**

def print_agent_data(self):


In [ ]:
def print_agent_data(self):

    print(f'TUMOR DATA id = {self.unique_id}')
    print(f'* Tumor Age = {150-self.lifespan}')
    print(f'* Proliferation Prob = {self.proliferation_prob}')
    print(f'* Death Prob = {self.death_prob}')
    print(f'* Direction sign {self.diff_sign}')
    print(f'* Distance to nearest Endothelial cell = {self.nearest_dist}')


## Basic Behaviour Methods

### Proliferation
Generate a new tumor cells by proliferation by the MainModel instance specific method *MainModel.generate_agents()* using the speicified arguments below. See the MainModel chapter for details. 

A new tumor cell is created adjacent to the same position as the current cell.


In [ ]:
#PROLIFERATION METHOD
def proliferate(self):
    try:
        self.model.generate_agents(Tumor_cells, "proliferate", 1, self.position, self.nearest_dist); 
    except Exception as e:
        print(f'Position: {self.position}')
        print(f"Error while generating Tumor cell from agent  {self.unique_id} {e}")
        pass

### Apoptosis
Induce apoptosis (programmed cell death) for the tumor cell. Called by itself and instances of the agent type M1.M1 that model macrophage type 1 cells. 

If statments handle edge cases where apoptosis has been induced in an exisitng model before it is called by itself for instance, commonly raising NoneType errors due to self.postion and self.pos being NoneType and or empty tuples.


In [ ]:
#APOPTOSIS METHOD:
def apoptosis(self): 
    if self.position == None:
        pass 
    elif self.pos != None and self.position != None:
        #print("DEAAD!")
        self.viable = False
        if self.position != None:
            self.model.grid.remove_agent(self);
            self.model.schedule.remove(self);

### Migration

Method that allows Tumor_cell instances to move by one model grid cell. adjacent to current position. 
To expand the model, migration could be modeled to increase across epithelial cells. This would aim to model tumors becoming malignant. 

In [ ]:
#MIGRATION
def migrate(self):
    """
    Move the tumor cell to a neighboring empty cell with a probability 
    defined by the migration probability.
    """
    #IN SITU MIGRATION???
    if random.randint(0,100) < 100*self.migration_prob:
        #print("TUMOR CELL MIGRATED")
        if self.pos != None:
            possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
            
            # Filter only empty positions
            empty_positions = [pos for pos in possible_steps if self.model.grid.is_cell_empty(pos)]

            #Pick an empty position if there are any
            if len(empty_positions) > 0:
                new_position = self.random.choice(empty_positions)
                self.model.grid.move_agent(self, new_position)

    #MIGRATION ACROSS BLOOD VESSLE
    #To be Continued       

## Model Environment Interacting Methods
The following methods interacts with the environment conditions to adjust parameters using setters and some of the above *basic behavioural methods* to adjust the probability of basic behaviour. 

### Hunger
Tracks nurient levels in the main model simulating nutrition dependnecy in the Tumor_cell instnace. 

Simulates the tumor cell's response to available nutrition.


The method adjusts various parameters based on the nutrient concentration (S) 
and the available nutrition cap in the model. It determines how hunger (low nutrition)
influences proliferation, death probability, and other characteristics of the tumor cell.

In [ ]:
#HUNGER (BROKEN)
def hunger(self):

    #COUNT CELLS
    total_cells = self.model.grid.width*self.model.grid.height + self.model.data_collection("count", "total") 
    #COUNT NUTRITION
    nutrition_cap = self.model.nutrition_cap
    #COUNT RATIO # Smaller ratio >>> decreased pro parameters, increased de parameters.
    S = nutrition_cap/(self.model.grid.width*self.model.grid.height) # nutrient_concentration
    nutrient_limit = 2
    
    if nutrition_cap > nutrient_limit:
        self.eat()
        if nutrition_cap >= nutrient_limit: # consumption rate, eat substrate
            if S >= nutrient_limit and self.nearest_dist > self.hypoxia_thresholds[1] and self.qs < self.qs_max:
                new_prol_prob = (1 * S)/(S + self.Ks_growth)
                self.set_proliferation_prob(new_prol_prob, "proportion")
    elif nutrition_cap < nutrient_limit:
        self.eat(0)
        self.set_angiogenesis_intensity(1.5)
        self.set_death_prob(1.2, 'proportion')
        self.set_proliferation_prob(0, "proportion")
        #print(nutrition_cap)

### Eat

Decreases the MainModel.nutrient_cap parameter; a parameter that the above *self.hunger()* tracks to set instance behaviour.

Is aimed to simulate substrate concentration cell specific consumption rate using parameters such as maximum consumption rate and half saturation constant to specify subtraction level from *MainModel.nutrient_cap* through the MainModel instance specfic method *MainModel.eat_nutrition()*.

In [ ]:
#EAT
def eat(self, *args):
    """
    Simulate the tumor cell eating nutrition based on the available concentration.

    Args:
        *args: Can specify a custom nutrient value.
    """

    S = self.model.nutrition_cap/(self.model.grid.width*self.model.grid.height) # nutrient_concentration
    if S > 0:
        self.qs = (self.qs_max)*(S)/(self.Ks_substrate*S)
        self.model.eat_nutrition(self.qs)
    if len(args) > 0 :
        val = args[0]
        self.model.eat_nutrition(val)

### Age

Increases the chance of death and decreases the proliferation probability of the Tumor_cell instance when called. 

**Usaege**

Is called in the *Tumor_cell.step()* method using the age specifc instance parameter *Tumor_cell.recParam1* to recursivly increase and decrease the death probability and proliferation probability respectively. 

In [ ]:
#AGE
def age(self):
    """
    Age the tumor cell, reducing its lifespan and modifying the proliferation 
    and death probabilities based on age.
    """
    if self.lifespan > 0:

        self.lifespan -= 1
        self.recParam1 += 0.001

        chance_of_death = self.recParam1
        decreased_prolif = self.recParam1*1.3
        
        self.death_prob += chance_of_death
        self.proliferation_prob -= decreased_prolif
    
    elif self.lifespan == 0:
        self.set_death_prob(1, "value")

## Tumor cell-Endothelial Cell Interaction

Model interactions between the Tumor_cell instance and its nearest endothelial cell set by calling *self.set_nearest_endo()*.

**Behaviour zones**

Models two distance based zones where interactions differ. If the distance between the instance and its nearest endothelial cell, specified in *self.nearest_dist* set by *self.set_nearest_endo()*. The zones are set by the parameters *self.hypoxia_threshold* which is a list which establishes the inter-turmo-endothelial cell distance limit for when the Turmo_cell instance experiences hypoxia. If *self.nearest_dist* is above under the first threshold value, no hypoxia is experienced and proliferaiton probability is set directly above the default value. If *self.nearest_dist* is above the threshold, *self.set_death_prob()* and *self.set_proliferation_prob* are used to dynamically multiply existing levels for their respective parameters with distance dependent factors. 

The endothelial cell specified by *self.nearest_endo* is also promoted to proliferate, the intensity of which is also dynamically set using a distnace dependent factor and the endothelial cell-instance specific method *Endothelial.targeted_proliferatoin()* See the *Endothelial* chapter for details. 

If the *self.nearest_dist* is above the third threshold value, the instance death probability is raised to simulate suffocation. 



In [ ]:

#TUMOR-ENDOTHELIAL CELL INTERACTIONS
def tumor_endo_interaction(self):
    """
    Perform interaction between the tumor cell and the nearest endothelial cell.

    This includes modifying the proliferation probability, death probability,
    and angiogenesis intensity based on the distance to the endothelial cell.
    """
    #print(f'Attempting tumor-endo interaction for agent: {self.unique_id}')
    #NOTATIONS
    best_dist = self.optimal_signal_dist
    curr_dist = self.nearest_dist
    threshold1 = self.hypoxia_thresholds[0]; threshold2 = self.hypoxia_thresholds[1]; threshold3 = self.hypoxia_thresholds[2]
    diff = self.diff
    diff_sign = self.diff_sign

    #INTERACTION ATTIBUTE PARAMETERS *FOR LOGISTIC ZONE*
    #  Death Intensity
    death_intensity = self.death_intensity
    delta_death_factor = death_intensity  * abs(curr_dist-threshold3)/threshold3 
    death_factor = delta_death_factor
    
    #  Tumor Proliferation Inhibition
    inhib_intensity = self.prolif_inhib_intensity 
    if curr_dist != 0:
        prolif_inhibition_level =  inhib_intensity*diff_sign * (curr_dist - threshold2)/curr_dist #relevant in the logistic zone
    elif curr_dist == 0:
        prolif_inhibition_level =  0

    proliferation_factor = 1-prolif_inhibition_level  # Higher inhibition level -> smaller factor -> smaller proliferation.
    
    #  Induction Level
    induct_intensity = self.angiogenesis_intensity 
    speed_dampening = self.optimal_signal_dist_significane #Lowers the significance of the optimal signal distance.
    if best_dist == curr_dist:
        induction_factor = 1
    else:
        induction_factor = induct_intensity * 1 / (1 + abs(best_dist-speed_dampening*curr_dist)) 
    
    self.set_nearest_endo('default'); #updates prev distance and new distance.
    
    #========ZONES========
    #DISCRETE ZONE
    if self.nearest_dist <= threshold1: #withing Lower bound
        self.set_proliferation_prob(0.15, "value")   #+20%
    elif threshold1 < self.nearest_dist <= threshold2:
        self.set_proliferation_prob("default")     #default

    #LOGISTIC ZONE
    elif self.nearest_dist > threshold2:
        if diff != 0:
            self.set_proliferation_prob(proliferation_factor, "proportion")
            if self.nearest_dist > threshold3:
                self.set_death_prob(death_factor, "proportion")
        self.nearest_endo.targeted_proliferation(self.position, induction_factor)          

    #PRINT AGENT DATA:
    #self.print_agent_data()

## Step

The step function specifies Tumor_cell instance specific behaviour at each step taken in the associated MainModel instance. 
To adjust behvaiour to environmental conditions, Tumor_cell.hunger() is called first, this sets new death-and proliferation probabilities. Then Tumro_cell.age() is called to adjust the same parameters based on the lifespan of the Tumor_cell instance. 
This is followed by migration and then Tumor_cell.tumor_endo_interaction().

After these methods have been called, the Tumor_cell instance will have new pareamter values for proliferation and death. The step function will thus try to call Tumor_cell.proliferate() and then Tumor_cell.apoptosis(). 

In [ ]:
#STEP 
def step(self):
    """
    Perform one step in the agent-based model.
    This involves moving, aging, proliferating, interacting with endothelial cells, and dying.
    """
    #print(f'TUMOR id: {self.unique_id}')
    #print(f'Initial proliferation_prob: {self.proliferation_prob}')
    #self.eat(10)
    self.hunger()
    #print(f'Hunger adjusted prolif prob: {self.proliferation_prob}')
    self.age()
    #print(f'Life Span: {self.lifespan}')
    #print(f'Age Adjusted Prolif Prob: {self.proliferation_prob}')
    self.migrate();
    self.tumor_endo_interaction();
    #print(f'Nearest distance: {self.nearest_dist}')
    #print(f'Distance adjusted Prolif prob: {self.proliferation_prob}')
    #print(f'Confirm Prolif Prob: {self.proliferation_prob}')
    #print(f'Death prob: {self.death_prob}')
    if random.randint(0,100) < 100*self.proliferation_prob:
        self.proliferate();
    if random.randint(0,100) < 100*self.death_prob:
        self.apoptosis()

## Potential for improvement

While functionally of the various functions has been confirmed, efficentcy modularity and better integration with paremters of the MainModel are warranted. For instance, the hunger-and eat operations were added at the end when clear logsitics limitations were noticied to not apply to growth. The only way that hypoxia was simulated was by the the Tumor_cell instane's distnace to their nearest endothelial cell. While this asumes the exisitance of a nutrient gradient, it does not directly simulte it. Even the exisitng nutrient count associated with the MainModel instnace does not necessarily form a gradient. Rather hunger due to nutrient deficiency and nutrient deficiency due to gradients are modelled spearaetly. 

To improve the model, a new class for nutrients could be created, in which the nutrients are spawned and dispursed away from endothelial cells. In additon, a new class for normal cells with a specifc substrate consumption rate is made for which consumption of nutrient instances that exisit in the same grid cell as themselves is simulated. Since MainModel uses the multigrid class, nutrient can pass cells that contain other agent instances. Those cells containing "full" cell agents, progress to migrate to the next cell in where the probability to be consumed would on a macroscopic scale be higher, thereby generating a concentration gradient perpendicular from the endothelial cells. 